In [ ]:
import json
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
import time
from tqdm.auto import tqdm
import json

In [2]:
from retrying import retry
import openai

@retry(stop_max_attempt_number=5, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def generate(prompt, history=[]):

    client = openai.OpenAI(api_key="sk-xxx", base_url="xxx")
    completion = client.chat.completions.create(
        model="xxx",
        messages = [{"role": "user", "content": prompt}],
        temperature = 0.
    )
    return completion.choices[0].message.content

In [3]:
with open("../../data/align/Align_Minos.json","r",encoding='utf-8-sig') as f:
    L = json.load(f)

In [4]:
input_data = []
for i in L:
    tmp = {}
    tmp['id'] = i['id']
    tmp['question'] = i['question']
    tmp['answer'] = i['answer']
    tmp['label'] = i['label']
    tmp['response'] = i['response']
    tmp['rank'] = i['rank']
    if tmp['label'] == 1:
        input_data.append(tmp)

In [ ]:
len(input_data)

In [6]:
prompt = '''<任务>
根据问题，从参考答案中提取出要点。每个要点应直接反映参考答案的具体内容。
</任务>

<格式要求>
1.请你将分析得到的要点，仿照下面的格式添加到一个python列表中，尽可能使用简洁的语言。
2.输出的python列表需要能够被json.loads()加载，示例如下：
{
    "key_points": ["要点1", "要点2", "要点3", ..., "要点n"]
}
</格式要求>

<注意>
1.分析得到的缺点，确保重点突出，但无需详细描述或解释，要保持简洁清晰。
2.任何不符合格式的输出都会导致系统的崩溃！
</注意>

<示例>
##输入
问题：高音单簧管和高音萨克斯的调性相同吗？如果相同，请说出他们的调性，如果不同，请分别说出他们的调性
参考答案：高音单簧管和高音萨克斯的调性不同。高音单簧管的调性通常为E♭，而高音萨克斯的调性则为B♭。
##输出
{
    "key_points": ["高音单簧管和高音萨克斯的调性不同","高音单簧管的调性为E♭","高音萨克斯的调性为B♭"]
}
</示例>

## 那么请你分析下面的回答，并输出符合格式的python列表
问题：{question} 
参考答案：{answer} 

输出：
'''

In [7]:
import re

def format_check(inputs, num=None):
    if isinstance(inputs,dict) and set(['key_points']):
        return True
    else:
        return False

# 定义一个函数，接受一个 markdown 字符串作为参数
def find_dicts(markdown):
    # 定义一个空列表，用于存储找到的 dict
    dicts = []
    
    if markdown == None:
        return dicts

    # 定义一个正则表达式，匹配 dict 的格式
    pattern = r"\{[^{}]*\}"
    # 使用 re.findall 方法，找出 markdown 字符串中所有匹配的子串
    matches = re.findall(pattern, markdown)
    # 遍历每个匹配的子串
    for match in matches:
        # 尝试将子串转换为 dict 类型，如果成功则添加到列表中
        try:
            d = eval(match)
            if isinstance(d, dict):
                dicts.append(d)
        except:
            # 如果转换失败，忽略该子串
            pass
    # 返回找到的 dict 列表
    return dicts

In [8]:
import traceback
import os
import json

import traceback
def deal(item,file_path):
    max_try_retry = 5
    try_num = 0
    content = prompt.replace('{question}',item['question']).replace('{answer}',item['answer'])
    for _ in range(max_try_retry):
        try:
            response = generate(content)
            tmps = find_dicts(response)[0]
            
            if format_check(tmps):
                item['points'] = tmps['key_points']
                with open(file_path, "a+", encoding="utf8") as f:
                    f.write(json.dumps(item, ensure_ascii=False) + "\n")
                    f.flush()
                break

        except Exception as e:
            str_e = str(e)
            if try_num== max_try_retry:
                break

            if 'InvalidRequestError' in str_e:
                if 'maximum context' in str_e:
                    break
                try_num += 1
                continue
            else:
                traceback.print_exc()

In [ ]:
file_path = "../../data/align/fact/align_points.jsonl"

if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        pass

with open(file_path,'r',encoding='utf-8') as f:
    L = f.readlines()

if len(L)>0:
    finish_id = [json.loads(i)['id'] for i in L]
else:
    finish_id = []

input_list = [i for i in input_data if i['id'] not in finish_id]

from tqdm.contrib import tzip
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib

num_worker = 32

with tqdm_joblib(desc="My calculation", total=len(input_list)) as progress_bar:
    Parallel(n_jobs=num_worker,prefer="threads")([delayed(deal)(x,file_path=file_path) for x in input_list])

In [10]:
with open(file_path, "r", encoding="utf-8") as f:
    L = f.readlines()
new_LL = [json.loads(i) for i in L]

with open("../../data/align/fact/align_points.json", "w", encoding="utf-8") as f:
    json.dump(new_LL, f, ensure_ascii=False, indent=4)